<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditTrainedModelsNotebooks/uofm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/uofm"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['preprocessUofM.ipynb', 'checkpoints', 'uofm_model.ipynb']


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 9.3 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=2

In [ ]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [ ]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/uofm.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 10429
Number of Utterances: 99470
Number of Conversations: 13469


In [ ]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()

In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)
temp.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
9q0ifj,1540092929,,AnonymousUser225,None,9q0ifj,357,None,1542634693,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",uofm,False,/r/uofm/comments/9q0ifj/go_blue/,,[]
81t5wn,1520112818,,dragpent,None,81t5wn,290,None,1522426820,0,None,uofm,False,/r/uofm/comments/81t5wn/15_michigan_beats_2_mi...,,[]
5qrvu9,1485662241,,stephen3141,None,5qrvu9,280,None,1489485314,0,None,uofm,False,/r/uofm/comments/5qrvu9/university_of_michigan...,,[]
5e8vyo,1479790064,,MrSplendiferous,None,5e8vyo,274,None,1484261504,0,None,uofm,False,/r/uofm/comments/5e8vyo/fuck_osu/,,[]
dq8zxll,1511469994,Bursley,Mongoose715,7f2b2g,7f2b2g,256,dq8zxll,1512940139,1,None,uofm,False,/r/uofm/comments/7f2b2g/traveling_to_ann_arbor...,,[]


In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)

posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]

posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]

conversation_ids_in = list(posts["conversation_id"])

coversation_ids = []
num_ids = 0
for id in conversation_ids_in:
  if num_ids == 20000:
      break
  dataframe = comments[comments["conversation_id"] == id]
  com_ids = list(dataframe.index)
  if com_ids != []:
    coversation_ids.append(id)
    num_ids = num_ids + 1

top_posts = posts.head(20000)
top_posts.head()

print(len(top_posts))


8018


In [ ]:
coversation_ids = coversation_ids[:20000] #not sure if needed 

In [ ]:
datasets_to_use = []
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

In [ ]:
import re
from nltk import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
train_ids, test_ids = train_test_split(coversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]
for index, id_list in enumerate(ids_total):
  for id in id_list:
      dataframe = comments[comments["conversation_id"] == id]
      post = posts.loc[id]['text']
      post = re.sub("\n", " ", post)
      post = re.sub("&gt;", "", post)
      post = re.sub("  ", " ", post)
      post = re.sub("&amp;", "", post)
      post = re.sub("#x200B;", "", post)
      post = re.sub(r'http\S+', '[url link]', post)
      post_tokens = word_tokenize(post)
      len_post = len(post_tokens)
      post_len[index].append(len_post)
      commented_here = ""
      com_ids = list(dataframe.index)
      if (len_post <= 512):
          running_sum = 0 # testing
          for id_c in com_ids:
            out_text = dataframe.loc[id_c]['text']
            out_text = str(out_text).strip()
            out_text = re.sub(r"\s", " ", out_text)
            out_text = re.sub(r"\n", " ", out_text)
            out_text = re.sub("&gt;", "", out_text) # added preprocess
            out_text = re.sub("  ", " ", out_text) # added preprocess
            out_text = re.sub("&amp;", "", out_text) # added preprocess
            out_text = re.sub("#x200B;", "", out_text) # added preprocess
            out_text = re.sub(r'http\S+', '[url link]', out_text) # added preprocess
            out_text_tokens = word_tokenize(out_text) # added this
            running_sum += len(out_text_tokens) # added this
            commented_here = commented_here + " " + out_text
            if running_sum > len_post: # changed this to 512
              break # added this
          comment_tokens = word_tokenize(commented_here)
          com_len[index].append(len(comment_tokens))
          post_utter = "<bos> " + post + " <eos> "
          comment_utter = "<bos>" + commented_here + " <eos> "
          posts_datasets_to_use[index].append(post_utter)
          comments_datasets_to_use[index].append(comment_utter)
print(posts_datasets_to_use[0][:10])
print(comments_datasets_to_use[0][:10])
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(post_len[0][0])
print(com_len[0][0])
print(len(post_len[0]))
print(len(com_len[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


["<bos> Hi everyone, I am a senior. Of course I was dumb and loaded a bunch of credits on for my last semester. Does anyone know any easy 1-2 credit classes? I'm a humanities major so the science-y stuff doesn't really appeal to me. I had the Astronomy one about aliens (106?) on my schedule but I heard it was really hard for a 1 credit class and you have to be good at math.  Please help me out : )  <eos> ", '<bos> Got waitlisted. Going to commit to UCLA in the meanwhile. Resident of Michigan. Chances of Michigan accepting off the waitlist?  <eos> ', "<bos> I'll be a freshmen next year and I just got my financial aid notice and it says I qualified for work study. I already live in Ann Arbor and currently work at the McDonald's on Stadium making about 8.50/hr but I'm planning to quit when school starts since taking the bus out there would probably be a pain - especially from North Campus. Has anyone done work study and do you think I should do it as a first year? Any suggestions for spec

In [ ]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 96))
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 75))

print(averages_post)
print(boundary_post)
print(averages_com)
print(boundary_com)

[88.16744278871741, 88.8876, 87.2208]
[301.0, 313.03999999999996, 295.03999999999996]
[114.40157798907546, 114.5156059991893, 114.12479674796748]
[143.0, 145.0, 143.25]


In [ ]:
train_tensors = []
val_tensors = []
test_tensors = []
train_com_tensors = []
val_com_tensors = []
test_com_tensors = []
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(len(tokenizer))

50260


In [ ]:
post_tensors_data = [train_tensors, val_tensors, test_tensors]
comments_tensors_data = [train_com_tensors, val_com_tensors, test_com_tensors]
# there doesn't seem to be a way to associate the two permanently, since tokenizer automatically removes it.
# maybe the exception is you reach into the other comment and remove?
# for instance, loop with like index: if it excepts, then: you go into the other coms dataset_to_use and remove that specific index.
# say in train, which is index = 0, and we find that at index1, post is too long. go into coms_dataset_to_use and delete that entry.
# i assume that it does the same thing for the other one. 
post_max = []
com_max = []
for index, dataset_tensors in enumerate(posts_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    post_tensors_data[index].append(text)
    #except:
    #print(index1) # watch for big tokens how to. like does this already truncate? 
    #del comments_datasets_to_use[index][index1]
    #del posts_datasets_to_use[index][index1] #maybe this one too for the next pass around
      # only problem is are the indexes reliable? like does index1 mean the same thing in both when you delete
      # and does a failed tokenizer actually do anything

for index, dataset_tensors in enumerate(comments_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    comments_tensors_data[index].append(text)

#for index, dataset_tensors in enumerate(posts_datasets_to_use):
#  for index1, text in enumerate(dataset_tensors):
#    encoded_input = tokenizer(text, return_tensors="pt", padding=True)
#    tensors_data[index].append(encoded_input)

In [ ]:
print(post_tensors_data[0][0])

<bos> Hi everyone, I am a senior. Of course I was dumb and loaded a bunch of credits on for my last semester. Does anyone know any easy 1-2 credit classes? I'm a humanities major so the science-y stuff doesn't really appeal to me. I had the Astronomy one about aliens (106?) on my schedule but I heard it was really hard for a 1 credit class and you have to be good at math.  Please help me out : )  <eos> 


In [ ]:

GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO = "448 Group: Jedi Order/models/uofm"
PATH_TO_PUT = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(os.listdir(PATH_TO_PUT))

448 Group: Jedi Order/models/uofm
['preprocessUofM.ipynb', 'checkpoints', 'uofm_model.ipynb']


In [ ]:
train_post_path = os.path.join(PATH_TO_PUT, "train_post_set.pickle")
val_post_path = os.path.join(PATH_TO_PUT, "val_post_set.pickle")
test_post_path = os.path.join(PATH_TO_PUT, "test_post_set.pickle")
train_com_path = os.path.join(PATH_TO_PUT, "train_com_set.pickle")
val_com_path = os.path.join(PATH_TO_PUT, "val_com_set.pickle")
test_com_path = os.path.join(PATH_TO_PUT, "test_com_set.pickle")

pickle.dump(posts_datasets_to_use[0], open(train_post_path, "wb"))
pickle.dump(posts_datasets_to_use[1], open(val_post_path, "wb"))
pickle.dump(posts_datasets_to_use[2], open(test_post_path, "wb"))
pickle.dump(comments_datasets_to_use[0], open(train_com_path, "wb"))
pickle.dump(comments_datasets_to_use[1], open(val_com_path, "wb"))
pickle.dump(comments_datasets_to_use[2], open(test_com_path, "wb"))

In [ ]:
posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
posts_datasets_to_use[1] = pickle.load(open(val_post_path, "rb"))
posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
comments_datasets_to_use[1] = pickle.load(open(val_com_path, "rb"))
comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
import tensorflow
### dataset.py ###
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, dataset_type: str = 'Train'):
    super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.posts)

  def get_labels(self) -> pd.Series:
    return self.comments
  
  def __getitem__(self, idx):
    post_encoding = self.posts[idx]
    comment_encoding = self.comments[idx]
    if self.dataset_type == 'Train':
      token = self.tokenizer.encode(post_encoding + 'TL;DR' + comment_encoding, return_tensors="pt", 
                                    padding="max_length", truncation=True)
    return token

In [ ]:
print(post_tensors_data[0][0])
print(comments_tensors_data[0][0])

<bos> Hi everyone, I am a senior. Of course I was dumb and loaded a bunch of credits on for my last semester. Does anyone know any easy 1-2 credit classes? I'm a humanities major so the science-y stuff doesn't really appeal to me. I had the Astronomy one about aliens (106?) on my schedule but I heard it was really hard for a 1 credit class and you have to be good at math.  Please help me out : )  <eos> 
<bos> I took ALA 115 this semester which was a one credit course about financial literacy (credit scores, banking, income inequality) taught by someone from the financial aid office. The outside work was about 1 page a week journal and a super simple group project at the end. I'm not sure if theyre offering it for winter term but it was almost no work and not even remotely difficult or stressful. Like total. Over the entire semester. Weekly quizzes are like 10 minutes and each of the 3 homeworks would be like 30-45 minutes <eos> 


In [ ]:
#print(len(tensors_data[index][0]["input_ids"][0]))
#print(len(tensors_data[index][0]["attention_mask"][0]))
#print(encoded_input["input_ids"].shape[1])
#print(encoded_input["attention_mask"].shape[1])

#train_to = zip(post_tensors_data[0], comments_tensors_data[0])
train_data = RedditDataset(post_tensors_data[0], comments_tensors_data[0])
#val_to = zip(post_tensors_data[1], comments_tensors_data[1])
val_data = RedditDataset(post_tensors_data[1], comments_tensors_data[1])
#test_to = zip(post_tensors_data[2], comments_tensors_data[2])
test_data = RedditDataset(post_tensors_data[2], comments_tensors_data[2])
#print(len(comments_tensors_data[0]))
#print(len(posts_datasets_to_use[0]))
#print(len(comments_datasets_to_use[0]))

In [ ]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
#from transformers import TextDataset
#print(len(train_dataset_to_use))
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "checkpoints")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    # do_train=True,
    num_train_epochs=1, # change this back to 2 
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy = "steps", #comment this
    eval_steps = 200,
    save_steps= 800,
    warmup_steps = 200,
    load_best_model_at_end = True, #comment this
    prediction_loss_only = True,
    )
# try and fix the above params as well
# convert dataset_to_use from a dictionary to a dataset type so that we can use it in the trainer
train_dataset_to_use_converted = torch.utils.data.DataLoader(train_dataset_to_use)
#print(type(train_dataset_to_use_converted))
test_dataset_to_use_converted = torch.utils.data.DataLoader(test_dataset_to_use)

#early_stop = EarlyStoppingCallback(5, 1.0) #comment this

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_data,
    eval_dataset=test_data
)

# depends on whether we want to use Trainer or not. I think using Trainer would be better if we get it to work,
# but I'm not sure how feasible it is. Additionally manual is kinda what we did in class
# ask about batching our data, and ask about num_labels. Batching our data in that dataloader part. like how to iterate over batches of data.
# idk if need to batch

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
200,No log,3.878907
400,No log,3.507209
600,10.699500,3.434445
800,10.699500,3.395724
1000,3.508500,3.362501
1200,3.508500,3.345872
1400,3.508500,3.319858
1600,3.423100,3.306096
1800,3.423100,3.298631
2000,3.413600,3.285068


TrainOutput(global_step=5442, training_loss=4.025570096622623, metrics={'train_runtime': 4870.6592, 'train_samples_per_second': 1.117, 'train_steps_per_second': 1.117, 'total_flos': 2843902476288000.0, 'train_loss': 4.025570096622623, 'epoch': 1.0})

In [ ]:
trainer.save_model()

VVVVVVVVrun if needed

In [ ]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints-preprocess-more")
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


^^^^^^run if needed 

In [ ]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
post_in_one = "I saw a squirrel today. I feel blessed. So fluffy. I saw it eat cute nuts."
post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_with_bos = "<bos>" + post_in + " TL;DR"
encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
    )
generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))
# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [ ]:
def make_sent(input):
    output = ""
    end_things = ".!?"
    last_end = 0
    i = 0
    while i < len(input):
        if input[i].isascii() or "'":
            if input[i] in end_things:
                output = output + input[i] + "\n"
                i = i + 1
                last_end = i
                if i + 1 < len(input):
                    if input[i] == " ":
                        i = i + 1
                        if i < len(input):
                            while input[i] == " ":
                                i = i + 1
                                if i >= len(input):
                                    break
                            output = output + " "  
            else:
                if i + 1 == len(input):
                    if input[i] == ",":
                        output = output + "."
                    else:
                        output = output + input[i]
                        output = output + "."
                else:
                    output = output + input[i]
                i = i + 1
    return output

In [ ]:
#https://python.plainenglish.io/use-python-to-split-sentences-into-separate-lines-17a577f9ddb3
print("Post: ")

post_out_now = make_sent(post_in)
print(post_out_now)

i = 1
for x in generated_sequences:
    out = x.split("TL;DR")
    comment = out[1]
    print("Comment {}: ".format(i))
    comment_out_now = make_sent(comment)
    print(comment_out_now)
    i = i+1
    print('\n')

Post: 
I want to go to hotel management school for rural bed and breakfasts.
 What experiences or classes should I seek out?

Comment 1: 
 Depends on the hotel and how well you can get into your hotel room.
 As for what I can recommend, I’ve had 3-4 hotelies over the past 3 years.
 I’ve even visited a hotel I own in NYC.
 If you can get into a hotel that's something I wouldn't recommend, but there are plenty of people in.


Comment 2: 
 A lot of hotelies also do the intro to hotel classes.
 I wouldn't recommend buying too many of those, just go through the hotel/vacation websites and talk with an older person (not necessarily the younger people), because you'll be more likely to find an older person in a hotel meeting to make a new group for you.



Comment 3: 
 You should learn how to hotel management in Cornell so you won't have to.
 I've taken one class with Peter Moore and he really got into the game quite quickly.
 I found him to be very helpful.
 He was a really nice guy, and.




Validation

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  post_with_bos = "<bos>" + post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(comments_datasets_to_use[1])
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(comments_datasets_to_use[1]):
  val_dataset[cnt]['reference_summary'] = item

for cnt, post in enumerate(post_tensors_data[1]):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge